# 1 Assignment 3

1. Generate a function named **beta_OLS**. This function must return the estimated beta using OLS. The inputs of this function should be `X`, `y`, and `intercept`. `X` (covariables) must be a **pd.DataFrame** and `y` (endog) must be a **pd.Series** and `intercept` can be `True` or `False`, by default `True`. When `intercept` is `False`, the estimated beta does not include **intercept**. Also, you must specify the type of your function's parameters and output and It must raise an error if the inputs do not meet the requirements. The function's output must be a **pd.DataFrame** (`n`, 1) where `n` is the total number of regressors ( it includes the **Intercept** when `intercept` is `True`). The column should be named as **Coef.** and the row index should be named as the original name of the columns in `X` input. Your results should look like `beta_OLS_output`. Apply your function to find $\widehat{\boldsymbol{\beta}}^{(OLS)}$ of the equation below. Use `data`.  **Hint: Use NumPy to generate the OLS beta and check [this link](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.set_index.html) to change the index name of a pd.Dataframe, [link](https://notebooks.githubusercontent.com/view/ipynb?browser=chrome&color_mode=auto&commit=69c80e1f2c1c268f0480a32932262201785a576c&device=unknown_device&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f616c6578616e6465727175697370652f4469706c6f6d61646f5f505543502f363963383065316632633163323638663034383061333239333232363232303137383561353736632f4c6563747572655f352f4c6563747572655f352e6970796e62&logged_in=true&nwo=alexanderquispe%2FDiplomado_PUCP&path=Lecture_5%2FLecture_5.ipynb&platform=windows&repository_id=427747212&repository_type=Repository&version=96##5.1.7.).**

$$
\widehat{\boldsymbol{\beta}}^{(OLS)} = \left( \mathbf{X}^\top \mathbf{X}\right)^{-1} \mathbf{X}^\top \mathbf{Y}
$$


$$
\begin{aligned} 
lnCT &= \beta_{0}+\beta_{q}lnq+ \beta_{qq}(lnq)^2+\beta_{q1}lnqlnp_1+\beta_{q2}lnqlnp_2+ \beta_{q3}lnqlnp_{3} +\beta_{1}lnp_1+\beta_{2}lnp_2+ \beta_{3}lnp_3 \\
& + \beta_{11}(lnp_{1})^2+ \beta_{22}(lnp_{2})^2+ \beta_{33}(lnp_{3})^2 + \beta_{12}lnp_{1}lnp_{2}+ \beta_{13}lnp_{1}lnp_{3}+\beta_{23}lnp_{2}lnp_{3} 
\end{aligned}
$$


In [163]:
import pandas as pd
import numpy as np
import pickle
import scipy.stats as stats

In [174]:
greene = pd.read_csv(r"../../_data/christensen_greene_f4.csv")
ct = greene.COST.values
q = greene.Q.values
p1 = greene.PL.values
p2 = greene.PF.values
p3 = greene.PK.values

In [141]:
file_to_open = open( r'../../_data/data_greene.pkl' , "rb" )
data = pickle.load( file_to_open )

In [175]:
X = data.iloc[ :, 1: ]

In [176]:
y = data.ln_cost

In [167]:
def calculator(x, y, intercept):
    lnp1 = (1/x*x)
    lnp2 = x*y
    result = lnp1*lnp2
    lnp3 = 1/x
    lnp4 = y
    result2 = lnp3 * lnp4
    if (intercept == TRUE):
        return result, lnp1, lnp2
    elif(intercept == FALSE):
        return result2, lnp3, lnp4
    else:
        print("Error, input does not meet the requirements")

In [120]:
file_to_open = open( r'../../_data/beta_OLS_output.pkl' , "rb" )
beta_OLS_output = pickle.load( file_to_open )
beta_OLS_output

,Coef.
Intercept,-76.259259
lnq,-1.080425
(lnq)^2,0.026489
(lnq)(lnp1),0.131041
(lnq)(lnp2),0.058652
(lnq)(lnp3),0.040144
lnp1,14.718292
lnp2,-0.894733
lnp3,6.380797
(lnp1)^2,-1.538527


In [177]:
def OLS_mat( X : pd.DataFrame, y : pd.Series, intercept = True ):
    
    X_n = X.copy()
    y_col = y.copy()
    
    if intercept == True:
        const = np.ones( len( X_n ) )
        X_n.insert(0, "Intercept", const)
    
    x_names = list( X_n.columns )
    X_n = X_n.to_numpy()
    y_col = y_col.to_numpy()
    
    x_inv = np.linalg.inv( np.dot( np.transpose( X_n ), X_n ) )
    
    return X_n, y_col, x_names, x_inv


In [178]:
def beta_OLS( X : pd.DataFrame, y : pd.Series, intercept = True ) -> pd.DataFrame:
    
    if not isinstance( X, pd.DataFrame ):
        raise TypeError( 'X is not a pd.DataFrame' )
    
    if not isinstance( y, pd.Series ):
        raise TypeError( 'y is not a pd.Series' )
    
    if not isinstance( intercept, bool ):
        raise TypeError( 'intercept is not a bool' )
        
    if len( X ) != len( y ):
        raise ValueError( 'X and y do not have the same number of rows' )
    
    X_n, y_col, x_names, x_inv = OLS_mat( X, y, intercept = intercept )
    
    coefs = np.dot( x_inv, np.dot( np.transpose( X_n ), y_col ) )
    coefs = pd.DataFrame( coefs, index = x_names, columns = [ 'Coef.' ] )
    
    return coefs

In [179]:
output = beta_OLS ( X, y )
output

,Coef.
Intercept,-76.259258
lnq,-1.080425
(lnq)^2,0.026489
(lnq)(lnp1),0.131041
(lnq)(lnp2),0.058652
(lnq)(lnp3),0.040144
lnp1,14.718292
lnp2,-0.894733
lnp3,6.380797
(lnp1)^2,-1.538527


In [180]:
output.info

<bound method DataFrame.info of                   Coef.
Intercept    -76.259258
lnq           -1.080425
(lnq)^2        0.026489
(lnq)(lnp1)    0.131041
(lnq)(lnp2)    0.058652
(lnq)(lnp3)    0.040144
lnp1          14.718292
lnp2          -0.894733
lnp3           6.380797
(lnp1)^2      -1.538527
(lnp2)^2      -0.073506
(lnp3)^2      -0.327414
(lnp1)(lnp2)   0.324566
(lnp1)(lnp3)  -1.144101
(lnp2)(lnp3)  -0.048049>

2. Generate a new function named **var_OLS**. This function must return the estimated variance for OLS. The inputs of this function should be `X`, `y`, and `intercept`. `X` (covariables) must be a **pd.DataFrame** and `y` (endog) must be a **pd.Series** and `intercept` can be `True` or `False`, by default `True`. When `intercept` is `False`, the estimated variance does not include **intercept** in the `X` regressor. Also, you must specify the type of your function's parameters and output and It must raise an error if the inputs do not meet the requirements. The output of the function should look like `var_OLS_output` where the columns and the index use the name of `X` columns.  Apply your function to find $\mathbb{V}{\rm ar} (\widehat{\boldsymbol{\beta}}^{(OLS)})$ of the equation below. Use greene data.  **Hint: Use numpy, `def` function, and `columns` method, [link](https://notebooks.githubusercontent.com/view/ipynb?browser=chrome&color_mode=auto&commit=69c80e1f2c1c268f0480a32932262201785a576c&device=unknown_device&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f616c6578616e6465727175697370652f4469706c6f6d61646f5f505543502f363963383065316632633163323638663034383061333239333232363232303137383561353736632f4c6563747572655f352f4c6563747572655f352e6970796e62&logged_in=true&nwo=alexanderquispe%2FDiplomado_PUCP&path=Lecture_5%2FLecture_5.ipynb&platform=windows&repository_id=427747212&repository_type=Repository&version=96##5.1.7.).**

$$
\widehat{\boldsymbol{\varepsilon}} = \mathbf{Y} - \mathbf{X} \widehat{\boldsymbol{\beta}}_{OLS}
$$


$$
\widehat{\sigma}^2_{OLS} = \dfrac{\widehat{\boldsymbol{\varepsilon}}^\top \widehat{\boldsymbol{\varepsilon}}}{N - (k+1)}
$$


$$
\begin{aligned}
\mathbb{V}{\rm ar} (\widehat{\boldsymbol{\beta}}^{(OLS)}) = 
\sigma^2 \left( \mathbf{X}^\top  \mathbf{X}\right)^{-1}
\end{aligned}
$$

$$
\begin{aligned} 
lnCT &= \beta_{0}+\beta_{q}lnq+ \beta_{qq}(lnq)^2+\beta_{q1}lnqlnp_1+\beta_{q2}lnqlnp_2+ \beta_{q3}lnqlnp_{3} +\beta_{1}lnp_1+\beta_{2}lnp_2+ \beta_{3}lnp_3 \\
& + \beta_{11}(lnp_{1})^2+ \beta_{22}(lnp_{2})^2+ \beta_{33}(lnp_{3})^2 + \beta_{12}lnp_{1}lnp_{2}+ \beta_{13}lnp_{1}lnp_{3}+\beta_{23}lnp_{2}lnp_{3} 
\end{aligned}
$$


In [109]:
file_to_open = open( r'../../_data/var_OLS_output.pkl' , "rb" )
var_OLS_output = pickle.load( file_to_open )
var_OLS_output

,Intercept,lnq,(lnq)^2,(lnq)(lnp1),(lnq)(lnp2),(lnq)(lnp3),lnp1,lnp2,lnp3,(lnp1)^2,(lnp2)^2,(lnp3)^2,(lnp1)(lnp2),(lnp1)(lnp3),(lnp2)(lnp3)
Intercept,1465.361133,0.192537,-0.011698,-0.026677,0.015949,0.039538,-262.932570,-8.392678,-128.192047,23.761713,-0.533889,7.081277,1.980507,21.804881,0.394870
lnq,0.192537,0.141034,0.000308,-0.013459,0.000161,-0.006089,-0.186223,0.148320,-0.090714,0.030641,0.000762,-0.001796,-0.030093,0.035093,-0.007141
(lnq)^2,-0.011698,0.000308,0.000006,-0.000036,-0.000005,-0.000011,0.001491,0.001052,0.000862,-0.000086,0.000026,-0.000120,-0.000198,-0.000028,-0.000097
(lnq)(lnp1),-0.026677,-0.013459,-0.000036,0.001466,-0.000168,0.000348,0.021561,-0.015127,0.005430,-0.003923,0.000038,0.000571,0.003700,-0.002282,-0.000170
(lnq)(lnp2),0.015949,0.000161,-0.000005,-0.000168,0.000653,-0.000189,-0.008293,0.003306,0.008035,0.001175,-0.000340,-0.000205,-0.000340,-0.000289,-0.002751
(lnq)(lnp3),0.039538,-0.006089,-0.000011,0.000348,-0.000189,0.000888,-0.000337,-0.008853,0.000268,0.000421,-0.000066,-0.000134,0.000204,-0.003074,0.004424
lnp1,-262.932569,-0.186223,0.001491,0.021561,-0.008293,-0.000337,50.194722,-1.253531,19.099543,-4.777971,0.140987,-0.840307,0.076832,-3.576317,0.264314
lnp2,-8.392678,0.148320,0.001052,-0.015127,0.003306,-0.008853,-1.253531,8.078742,-0.214871,0.323286,-0.117293,-0.066294,-1.370100,0.398963,-0.734776
lnp3,-128.192047,-0.090714,0.000862,0.005430,0.008035,0.000268,19.099543,-0.214871,20.513714,-1.410427,0.047330,-1.526100,0.072458,-3.111107,-0.145182
(lnp1)^2,23.761713,0.030641,-0.000086,-0.003923,0.001175,0.000421,-4.777971,0.323286,-1.410427,0.479422,-0.012318,0.045414,-0.053651,0.281523,-0.026624


In [181]:
def var_OLS( X : pd.DataFrame, y : pd.Series, intercept = True ) -> pd.DataFrame:
    
    coefs = beta_OLS( X, y, intercept = intercept ).iloc[ :, 0 ].to_numpy()
    X_n, y_col, x_names, x_inv = OLS_mat( X, y, intercept = intercept )
    
    y_hat = np.dot( X_n, coefs )
    res_vec = y_col - y_hat
    sigma2 = np.dot( np.transpose( res_vec ), res_vec ) / ( len( y ) - len( coefs ) )
    
    ols_vcov = x_inv * sigma2
    ols_vcov = pd.DataFrame( ols_vcov, index = x_names, columns = x_names )
    
    return ols_vcov

In [182]:
var_OLS( X , y )

,Intercept,lnq,(lnq)^2,(lnq)(lnp1),(lnq)(lnp2),(lnq)(lnp3),lnp1,lnp2,lnp3,(lnp1)^2,(lnp2)^2,(lnp3)^2,(lnp1)(lnp2),(lnp1)(lnp3),(lnp2)(lnp3)
Intercept,1465.361103,0.192537,-0.011698,-0.026677,0.015949,0.039538,-262.932564,-8.392679,-128.192044,23.761712,-0.533889,7.081277,1.980507,21.804880,0.394870
lnq,0.192537,0.141034,0.000308,-0.013459,0.000161,-0.006089,-0.186223,0.148320,-0.090714,0.030641,0.000762,-0.001796,-0.030093,0.035093,-0.007141
(lnq)^2,-0.011698,0.000308,0.000006,-0.000036,-0.000005,-0.000011,0.001491,0.001052,0.000862,-0.000086,0.000026,-0.000120,-0.000198,-0.000028,-0.000097
(lnq)(lnp1),-0.026677,-0.013459,-0.000036,0.001466,-0.000168,0.000348,0.021561,-0.015127,0.005430,-0.003923,0.000038,0.000571,0.003700,-0.002282,-0.000170
(lnq)(lnp2),0.015949,0.000161,-0.000005,-0.000168,0.000653,-0.000189,-0.008293,0.003306,0.008035,0.001175,-0.000340,-0.000205,-0.000340,-0.000289,-0.002751
(lnq)(lnp3),0.039538,-0.006089,-0.000011,0.000348,-0.000189,0.000888,-0.000337,-0.008853,0.000268,0.000421,-0.000066,-0.000134,0.000204,-0.003074,0.004424
lnp1,-262.932564,-0.186223,0.001491,0.021561,-0.008293,-0.000337,50.194721,-1.253531,19.099542,-4.777971,0.140987,-0.840307,0.076832,-3.576317,0.264314
lnp2,-8.392679,0.148320,0.001052,-0.015127,0.003306,-0.008853,-1.253531,8.078742,-0.214871,0.323286,-0.117293,-0.066294,-1.370100,0.398963,-0.734776
lnp3,-128.192045,-0.090714,0.000862,0.005430,0.008035,0.000268,19.099542,-0.214871,20.513714,-1.410427,0.047330,-1.526100,0.072458,-3.111107,-0.145182
(lnp1)^2,23.761712,0.030641,-0.000086,-0.003923,0.001175,0.000421,-4.777971,0.323286,-1.410427,0.479422,-0.012318,0.045414,-0.053651,0.281523,-0.026624


3. Generate a new function named as `reg_OLS` that uses your previos functions `beta_OLS` and `var_OLS` to return an ouput similar to `reg_OLS_output`. The inputs of this function should be `X`, `y`, and `intercept`. `X` (covariables) must be a **pd.DataFrame** and `y` (endog) must be a **pd.Series** and `intercept` can be `True` or `False`, by default `True`. When `intercept` is `False`, the output does not include **intercept**. Also, you must specify the type of your function's parameters and output and It must raise an error if the inputs do not meet the requirements. The ouput of this function should look like `reg_OLS_ouput` Apply your function to the equation below. Use greene data.**Hint: Use numpy, `def` function, and `columns` method, [link](https://notebooks.githubusercontent.com/view/ipynb?browser=chrome&color_mode=auto&commit=69c80e1f2c1c268f0480a32932262201785a576c&device=unknown_device&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f616c6578616e6465727175697370652f4469706c6f6d61646f5f505543502f363963383065316632633163323638663034383061333239333232363232303137383561353736632f4c6563747572655f352f4c6563747572655f352e6970796e62&logged_in=true&nwo=alexanderquispe%2FDiplomado_PUCP&path=Lecture_5%2FLecture_5.ipynb&platform=windows&repository_id=427747212&repository_type=Repository&version=96##5.1.7.).**

$$
\begin{aligned} 
lnCT &= \beta_{0}+\beta_{q}lnq+ \beta_{qq}(lnq)^2+\beta_{q1}lnqlnp_1+\beta_{q2}lnqlnp_2+ \beta_{q3}lnqlnp_{3} +\beta_{1}lnp_1+\beta_{2}lnp_2+ \beta_{3}lnp_3 \\
& + \beta_{11}(lnp_{1})^2+ \beta_{22}(lnp_{2})^2+ \beta_{33}(lnp_{3})^2 + \beta_{12}lnp_{1}lnp_{2}+ \beta_{13}lnp_{1}lnp_{3}+\beta_{23}lnp_{2}lnp_{3} 
\end{aligned}
$$


$$
\widehat{\boldsymbol{\varepsilon}} = \mathbf{Y} - \mathbf{X} \widehat{\boldsymbol{\beta}}_{RLS}
$$


$$
\widehat{\sigma}^2_{RLS} = \dfrac{\widehat{\boldsymbol{\varepsilon}}^\top \widehat{\boldsymbol{\varepsilon}}}{N - (k+1 - M)}
$$

In [18]:
file_to_open = open( r'../../_data/reg_OLS_output.pkl' , "rb" )
reg_OLS_output = pickle.load( file_to_open )
reg_OLS_output

,Coef.,Std.Err.,t,P>|t|,[0.025,0.975]
Intercept,-76.259259,38.280036,-1.992142,4.825953e-02,-151.927107,-0.591411
lnq,-1.080425,0.375545,-2.876952,4.630885e-03,-1.822762,-0.338088
(lnq)^2,0.026489,0.002357,11.240060,2.112040e-21,0.021831,0.031148
(lnq)(lnp1),0.131041,0.038284,3.422854,8.084865e-04,0.055365,0.206716
(lnq)(lnp2),0.058652,0.025549,2.295650,2.315002e-02,0.008149,0.109155
(lnq)(lnp3),0.040144,0.029791,1.347518,1.799447e-01,-0.018744,0.099033
lnp1,14.718292,7.084823,2.077440,3.954949e-02,0.713777,28.722807
lnp2,-0.894733,2.842313,-0.314790,7.533796e-01,-6.513111,4.723645
lnp3,6.380797,4.529207,1.408811,1.610617e-01,-2.572051,15.333645
(lnp1)^2,-1.538527,0.692403,-2.222011,2.785341e-02,-2.907195,-0.169860


4. Generate a new function named as `restricted_OLS` that uses your previos functions `beta_OLS` and `var_OLS` to return an ouput similar to `reg_OLS_output`. The inputs of this function should be `X`, `y`,  `L`, `r`, and `intercept`. `X` (covariables) must be a **pd.DataFrame** and `y` (endog) must be a **pd.Series**,  `L` (coef) must be a **np.array**, `r` must be a **np.array**,  and `intercept` can be `True` or `False`.  When `intercept` is `False`, the estimated beta does not include **intercept**. Also, you must specify the type of your function's parameters and output and It must raise an error if the inputs do not meet the requirements. The ouput of this function should look like `restricted_OLS_ouput`. Apply your function to the equation below. Use greene data.
 **Hint: Use numpy, `def` function, and `columns` method, [link](https://notebooks.githubusercontent.com/view/ipynb?browser=chrome&color_mode=auto&commit=69c80e1f2c1c268f0480a32932262201785a576c&device=unknown_device&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f616c6578616e6465727175697370652f4469706c6f6d61646f5f505543502f363963383065316632633163323638663034383061333239333232363232303137383561353736632f4c6563747572655f352f4c6563747572655f352e6970796e62&logged_in=true&nwo=alexanderquispe%2FDiplomado_PUCP&path=Lecture_5%2FLecture_5.ipynb&platform=windows&repository_id=427747212&repository_type=Repository&version=96##5.1.7.).**

$$
\hat{\boldsymbol{\beta}}^{(RLS)} =\hat{\boldsymbol{\beta}}^{(OLS)} - \left( \boldsymbol{X}^\top \boldsymbol{X} \right)^{-1} \boldsymbol{L}^\top \left( \boldsymbol{L}\left( \boldsymbol{X}^\top \boldsymbol{X} \right)^{-1} \boldsymbol{L}^\top \right)^{-1} \left( \boldsymbol{L}\widehat{\boldsymbol{\beta}}^{(OLS)} - \boldsymbol{r} \right)
$$

$$
\begin{aligned}
\mathbb{V}{\rm ar} \left(\widehat{\boldsymbol{\beta}}^{(RLS)} \right) 
&= \sigma^2 \left( \mathbf{X}^\top  \mathbf{X}\right)^{-1} - \sigma^2 \left( \mathbf{X}^\top  \mathbf{X}\right)^{-1}  \mathbf{L}^\top \left( \mathbf{L}\left( \mathbf{X}^\top \mathbf{X} \right)^{-1} \mathbf{L}^\top \right)^{-1}  \mathbf{L} \left( \mathbf{X}^\top  \mathbf{X}\right)^{-1}
\end{aligned}
$$

$$
\begin{aligned} 
lnCT &= \beta_{0}+\beta_{q}lnq+ \beta_{qq}(lnq)^2+\beta_{q1}lnqlnp_1+\beta_{q2}lnqlnp_2+ \beta_{q3}lnqlnp_{3} +\beta_{1}lnp_1+\beta_{2}lnp_2+ \beta_{3}lnp_3 \\
& + \beta_{11}(lnp_{1})^2+ \beta_{22}(lnp_{2})^2+ \beta_{33}(lnp_{3})^2 + \beta_{12}lnp_{1}lnp_{2}+ \beta_{13}lnp_{1}lnp_{3}+\beta_{23}lnp_{2}lnp_{3} 
\end{aligned}
$$

ST: 

<br>
$$
\begin{aligned} 
\beta_{1} + \beta_{2} + \beta_{3} &= 1 \\
\beta_{q1} + \beta_{q2} + \beta_{q3} &= 0 \\
\beta_{11} + \beta_{12} + \beta_{13} &= 0 \\
\beta_{21} + \beta_{22} + \beta_{23} &= 0 \\
\beta_{31} + \beta_{32} + \beta_{33} &= 0 \\
\beta_{ij} = \beta_{ji}
\end{aligned}
$$

In [19]:
L = np.array( ( [ 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0 ], 
              [ 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0 ], 
              [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0 ], 
              [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1 ], 
              [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1 ] ) )

r = np.array(  [  1,  0,  0,  0,  0  ]  ).reshape( -1 , 1 )

In [23]:
file_to_open = open( r'../../_data/restricted_OLS_output.pkl' , "rb" )
restricted_OLS_output = pickle.load( file_to_open )
restricted_OLS_output

,Coef.,Std.Err.,t,P>|t|,[0.025,0.975]
Intercept,-7.111200,2.755866,-2.580387,1.091364e-02,-12.560384,-1.662016
lnq,0.461368,0.119721,3.853679,1.776124e-04,0.224642,0.698094
(lnq)^2,0.029978,0.002121,14.134763,1.312279e-28,0.025784,0.034171
(lnq)(lnp1),-0.004354,0.022880,-0.190311,8.493452e-01,-0.049596,0.040887
(lnq)(lnp2),0.036778,0.024201,1.519668,1.308823e-01,-0.011075,0.084631
(lnq)(lnp3),-0.032424,0.021419,-1.513780,1.323685e-01,-0.074775,0.009928
lnp1,0.215566,1.015206,0.212338,8.321570e-01,-1.791804,2.222936
lnp2,0.402923,0.554162,0.727085,4.684055e-01,-0.692824,1.498670
lnp3,0.381511,0.531316,0.718048,4.739416e-01,-0.669063,1.432084
(lnp1)^2,-0.008712,0.140961,-0.061806,9.508067e-01,-0.287435,0.270011


5. Generate a new class named `RegClass`. The initial attributes of your class must be `X`, `y`, and `intercept`. `X` (covariables) must be a **pd.DataFrame** and `y` (endog) must be a **pd.Series** and `intercept` can be `True` or `False`, by default `True`. Also, you must specify the type of your class' parameters, and output and It must raise an error if the inputs do not meet the requirements.  This class should have two methods: `reg_OLS` and `restricted_OLS` (You can add more if you think it is necessary). These methods should return the same output as the previously defined functions with the same name. The method `reg_OLS` does not need an extra parameter to be executed. After executing the method `reg_OLS`, any user should access the following attributes: `beta_OLS` and `var_OLS`. They should look like `beta_OLS_output` and `var_OLS_output`. The `restricted_OLS` method needs two extra parameters before being executed, `L` and `r`. They refer to the restrictions. After running the method `restricted_OLS`, any user should access the following attributes: `beta_OLS`, `var_OLS`, `rest_beta_OLS`, and `rest_var_OLS`. These attributes refer to the beta and variance estimated using OLS and restricted OLS. They should look like  `beta_OLS_output` and `var_OLS_output`. Fix all the methods and attributes. No user should be able to change them. **Hint: Use [lecture 6](https://notebooks.githubusercontent.com/view/ipynb?browser=chrome&color_mode=auto&commit=69c80e1f2c1c268f0480a32932262201785a576c&device=unknown_device&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f616c6578616e6465727175697370652f4469706c6f6d61646f5f505543502f363963383065316632633163323638663034383061333239333232363232303137383561353736632f4c6563747572655f362f4c6563747572655f362e6970796e62&logged_in=true&nwo=alexanderquispe%2FDiplomado_PUCP&path=Lecture_6%2FLecture_6.ipynb&platform=windows&repository_id=427747212&repository_type=Repository&version=96#6.3.).**